In [51]:
!pip install langchain_community==0.2.6 tiktoken fastembed==0.3.2 langchainhub chromadb langchain bs4 langchain-chroma langchain-groq

In [69]:
import os
from dotenv import load_dotenv

load_dotenv()

True

In [70]:
LANGCHAIN_API_KEY=os.getenv('LANGCHAIN_API_KEY')
LANGCHAIN_TRACING_V2=os.getenv('LANGCHAIN_TRACING_V2')
LANGCHAIN_ENDPOINT=os.getenv('LANGCHAIN_ENDPOINT')
GROQ_API_KEY=os.getenv('GROQ_API_KEY')

## RAG System: Indexing, Retrieval, and Generation

In [57]:
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
import numpy as np

In [3]:
# documents

doc1 = "My name is Zohaib, and I like ice-cream"
doc2 = "Grass is greener on the other side"

query = "Zohaib likes green color and ice-cream"

In [9]:
# using FastEmbed embeddings to embed the documents
# using np.sqeeze to get rid of 1D arrays

embd = FastEmbedEmbeddings()
query_result = np.squeeze(embd.embed_query(query))
doc1_embd = np.squeeze(embd.embed_documents(doc1))
doc2_embd = np.squeeze(embd.embed_documents(doc2))
len(query_result)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

384

In [10]:
# function for finding cosine similarity b/w two vectors

def cosine_similarity(vec1, vec2):
    dot_prod = np.dot(vec1, vec2)
    norm_vec1 = np.linalg.norm(vec1)
    norm_vec2 = np.linalg.norm(vec2)
    return dot_prod / (norm_vec1 * norm_vec2)

In [13]:
# calculating similarities

sim_2 = cosine_similarity(query_result, doc2_embd)
sim_1 = cosine_similarity(query_result, doc1_embd)

if cosine_similarity(query_result, doc2_embd) > cosine_similarity(query_result, doc1_embd):
    print("Document 2 is more relevant")
else:
    print("Document 1 is more relevant")

print(f"Similarity of Document 1 and Query: {sim_1}")
print(f"Similarity of Document 2 and Query: {sim_2}")

Document 1 is more relevant
Similarity of Document 1 and Query: 0.8165447261910522
Similarity of Document 2 and Query: 0.5760569530933125


#### Indexing

In [61]:
import bs4
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [21]:
# loading blog, which will be our document

from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
blog_docs = loader.load()

In [24]:
# splitting the document using text splitter

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)

splits = splitter.split_documents(blog_docs)

In [32]:
# initializing vector store

vectordb = Chroma(embedding_function=FastEmbedEmbeddings())

retriever = vectordb.as_retriever()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [33]:
retriever.add_documents(splits) # indexing documents

['fc7543f9-1619-4351-b545-21538cce824a',
 '1539c176-d0c6-4676-a6d4-ae9eddb033a6',
 'c1d00841-ff09-4e09-9d18-cbd881581fa0',
 '7837d154-2478-4bb0-a0a3-d8ac5c9ffa51',
 'd0e6d4db-c822-4a20-9eeb-01bbc49ecddd',
 'ba251094-2b89-476a-9a00-20130bf41829',
 '4d8b2e5b-f5b4-4318-89c4-d9202ec9d971',
 'c7d2b346-0d13-4a07-b2b1-392f7972b9c0',
 'ee3a6f9f-79ca-47ed-9ed9-a5608918d7ee',
 '4b651d88-1f98-40b7-8b17-29f93a846382',
 'ab725a8a-c324-4439-9676-c4b94b26920d',
 'af9cf2af-c64e-4335-b103-1457e7f45576',
 '4057025d-a5a8-4ba8-8711-26d2f6ed4151',
 'd35dd397-5976-470f-9240-9e91069cbb38',
 'cb8cd522-37dc-4691-bdb2-ecd8dab72637',
 'e19391f1-bc60-4a0d-bddd-e14d87cf99a2',
 'fad20f5a-492b-43f7-8eac-e928d2f42820',
 'edcb58d6-94c3-45e0-9403-0fa39af5a85f',
 'b6baf0aa-471e-4ea1-8095-3ad36cf06bfd',
 '1db0f853-e01b-4a6d-b8ae-a34897be6459',
 '13722423-ede6-4084-a375-40fe657342f3',
 'e511c208-4d5b-4dfa-b9c7-f19a9668a0c5',
 '84868319-9b5a-47d8-b7e4-1a46bc3e4892',
 '392478ec-6771-4627-a6f3-735c8117d4a9',
 '989e2ff9-fd9d-

#### Retrieval

In [42]:
#initializing vectorstore

vectordb = Chroma(embedding_function=FastEmbedEmbeddings())

retriever = vectordb.as_retriever(search_kwargs={'k':1}) # k nearest documents will be retrieved
retriever.add_documents(splits)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

['299159eb-36b2-46af-b3fe-da82d05eb62c',
 '889181fb-0f39-4c58-8e40-26eb566a25b3',
 'af0b0b9a-fc7c-4689-beef-38bd64dc6f64',
 'ce77de98-44b8-4776-95dc-5fa7d0bc0566',
 '7e548066-9482-4a6e-b0cf-084f491c5290',
 '329f977c-3dff-4c20-86c2-a27e7fe172db',
 '66be261c-da4e-4cf4-b6af-da9b620d4afa',
 'ea18a6bf-1b6b-41ce-8a0f-8a4e5df1073c',
 '30173a09-dfbf-4742-8d3d-c92fa1313442',
 'b38554fd-5d05-4cf1-8a48-1eb1f9400e01',
 '3d564ace-f8c9-4e3b-9850-84ddc32f3353',
 'a66c121d-923e-4055-a567-4a1d129ea39a',
 'd08844a0-39a8-4c96-8d13-fa90950778f8',
 'f76c2600-bccf-4c81-ad22-89007b79ab49',
 '082e83e5-204c-4cff-a12b-fa5ff6b247fb',
 '502d7d70-86cb-4def-8d76-36d5e0ecdb53',
 'a5354517-82f1-4f3d-876f-cfdef51e6d90',
 '707cb7f8-f908-42c5-9fb0-2bb613cc8584',
 '6fa69977-8672-4596-a7dc-9674e3e885e5',
 '03010c85-343d-4524-be89-08ece5620913',
 'f006824f-3457-45ad-bfe2-c7b0310c6595',
 '711893cb-1778-4eb8-a4eb-9442773ca9f8',
 '61acaf5c-a723-49f5-a0b2-db504096bc0d',
 'a6443253-85eb-46de-a096-2f1dbf9183c0',
 'c0de6145-526e-

In [43]:
doc = retriever.get_relevant_documents('What is Task Decomposition') # performing semantic search to get relevant doc

In [41]:
len(doc) # 1 doc retrieved, as expected, since k is 1

1

In [50]:
print(doc[0].page_content)

Task decomposition can be done (1) by LLM with simple prompting like "Steps for XYZ.\n1.", "What are the subgoals for achieving XYZ?", (2) by using task-specific instructions; e.g. "Write a story outline." for writing a novel, or (3) with human inputs.


#### Generation

In [83]:
from langchain_groq import ChatGroq
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [63]:
# creating the prompt template

template = """Answer the question based on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template=template)

In [77]:
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0) # we will be using the model Llama 3.2 Groq

In [80]:
# defining a chain

chain = prompt | llm

In [79]:
# invoke the chain, by passing in the context and the question

chain.invoke({'context':doc, 'question':"What is Task Decomposition"}).content

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or steps.'

In [84]:
# now we will automate all this work with a RAG chain

rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [87]:
rag_chain.invoke("What is Task Decomposition?")

'Task decomposition is the process of breaking down a complex task into smaller, more manageable subtasks or subgoals.'